In [73]:
import main_module as m
import zoho_filter_module as fil
import pandas as pd
import re
import numpy as np

%load_ext autoreload
%autoreload 2

cols = ['Record Id',	
'Created Time',
'Created Date',	
'First Name',	
'Last Name',	
'Email',	
'Phone',	
'Mobile',	
'Birthdate',	
'Year of Birth',
'Street',	
'Street2',	
'City',	
'State',	
'Country',	
'Nationality',
'Booking Status', 
'BookRef',
'Departure Date',	
'Arrival Date',	
'Card Type',	
'Marital Status',	
'Lead Brand',	
'Lead Sub-Brand',	
'Lead Source',	
'Lead Locations',	
'Lead Source Description',	
'Lead Regions',
'Lead Status',
'Email Opt Out',	
'Opt In',
'GDPR Compliant',
'Website',
'Comments',
'Do you live in Bali?']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
path = r'C:\Users\fajar.fatoni\Documents\Python\Data\Wedding Leads from Europe Resorts.xlsx'
# zoho_df = pd.read_csv(path, dtype='string')
zoho_df = pd.read_excel(path, dtype='string', sheet_name='KSH')

zoho_df.reset_index(drop=True, inplace=True)
count_filter_awal = zoho_df.shape[0]

# mengisi kolom <NA> dengan value string 'empty' supaya mudah mengecek data member dan booking dengan isin
zoho_df.fillna('empty', inplace=True)

zoho_df['Mobile'] = 'empty'
# zoho_df['Phone'] = 'empty'
zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Mobile
0,zcrm_3584690000475129897,2024-01-02 03:06:22,2024-01-01 00:00:00,Bethany Roberts,Bethany Roberts,broberts2611@gmail.com,447802613306,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
1,zcrm_3584690000475363902,2024-01-03 23:45:38,2024-01-03 00:00:00,Emmy,Emmy,emmybadham@btinternet.com,447817178812,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
2,zcrm_3584690000475525104,2024-01-05 06:03:36,2024-01-04 00:00:00,Tikeya Holloway,Tikeya Holloway,tikeyaholloway@gmail.com,447582540744,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
3,zcrm_3584690000475544144,2024-01-05 01:52:12,2024-01-04 00:00:00,Jason And Emma,Jason And Emma,jasesanders1993@yahoo.com,447940033370,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
4,zcrm_3584690000475544281,2024-01-05 06:32:32,2024-01-04 00:00:00,Ben Caldwell,Ben Caldwell,caldwellb395@gmail.com,447934857239,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,empty,empty,empty,M,-,marina.atkins@hotmail.co.uk,447961502336,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1478,empty,empty,empty,Samantha,-,sam.laister5@gmail.com,447912877649,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1479,empty,empty,empty,Spencer,Hughes-Morris,Spencer.d.hughes.morris@gmail.com,447826640590,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1480,empty,empty,empty,David,Ortega,me@davidortega.co.uk,447776552414,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty


In [75]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04eXpvS1NXTWF2RVE\_BOOKING\Booking By Year\FlyBuy Booking Data (BGAU & BGID & KE).xlsx'
df_booking = pd.read_excel(path, dtype='string', sheet_name='BOOKING_BGAU_BGID_KE')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


# ====================== Awal Filter Umum ====================== 

In [76]:
# data phone, mobile dam email member yang ada di master data (wash member)

m.clean_number(zoho_df, 'Phone')
m.clean_number(zoho_df, 'Mobile')
m.replace_not_valid_to_empty(zoho_df, 'Phone')
m.replace_not_valid_to_empty(zoho_df, 'Mobile')
zoho_df['Email'] = m.lowercase(zoho_df, 'Email')
m.clean_space(zoho_df, 'Email')
m.clean_not_valid_email_format(zoho_df, 'Email')

m.clean_number(df_booking, 'Phone1')
m.clean_number(df_booking, 'Mobile')
df_booking['Email'] = m.lowercase(df_booking, 'Email')
m.clean_space(df_booking, 'Email')
m.clean_not_valid_email_format(df_booking, 'Email')

m.clean_number(df_phone, 'Mobile')

df_email['Email'] = m.lowercase(df_email, 'Email')
m.clean_space(df_email, 'Email')
m.clean_not_valid_email_format(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:71: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)
c:\Users\fajar.fatoni\Documents\Python\Task Offline\zoho_data_app\main_module.py:72: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setti

'Email'

In [77]:
count_deleted_wrong_format = m.delete_email_wrong_format(zoho_df, 'Email')
count_after_deleted_wrong_format = zoho_df.shape[0]

# ====================== Akhir Filter Umum ====================== 

In [78]:
# hitung jumlah duplicate
count_duplicate_general = zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])].shape[0]

zoho_df.loc[zoho_df.duplicated(subset=['Phone','Mobile','Email'])]

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Mobile


In [79]:
zoho_df.drop_duplicates(subset=['Phone','Mobile','Email'], inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = zoho_df.shape[0]

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Mobile
0,zcrm_3584690000475129897,2024-01-02 03:06:22,2024-01-01 00:00:00,Bethany Roberts,Bethany Roberts,broberts2611@gmail.com,447802613306,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
1,zcrm_3584690000475363902,2024-01-03 23:45:38,2024-01-03 00:00:00,Emmy,Emmy,emmybadham@btinternet.com,447817178812,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
2,zcrm_3584690000475525104,2024-01-05 06:03:36,2024-01-04 00:00:00,Tikeya Holloway,Tikeya Holloway,tikeyaholloway@gmail.com,447582540744,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
3,zcrm_3584690000475544144,2024-01-05 01:52:12,2024-01-04 00:00:00,Jason And Emma,Jason And Emma,jasesanders1993@yahoo.com,447940033370,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
4,zcrm_3584690000475544281,2024-01-05 06:32:32,2024-01-04 00:00:00,Ben Caldwell,Ben Caldwell,caldwellb395@gmail.com,447934857239,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,empty,empty,empty,M,-,marina.atkins@hotmail.co.uk,447961502336,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1478,empty,empty,empty,Samantha,-,sam.laister5@gmail.com,447912877649,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1479,empty,empty,empty,Spencer,Hughes-Morris,spencer.d.hughes.morris@gmail.com,447826640590,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1480,empty,empty,empty,David,Ortega,me@davidortega.co.uk,447776552414,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty


In [80]:
# Cek leads test
test_leads = zoho_df.loc[
    
        zoho_df['Email'].str.contains('^test@', flags=re.I, regex=True) |
        zoho_df['Email'].str.contains('\+test', flags=re.I, regex=True) |
    
        zoho_df['First Name'].str.contains('^test$', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('^test$', flags=re.I, regex=True) |

        zoho_df['First Name'].str.contains('test 1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test 2', flags=re.I, regex=True)|
        zoho_df['First Name'].str.contains('test1', flags=re.I, regex=True) |
        zoho_df['Last Name'].str.contains('test2', flags=re.I, regex=True)|

        zoho_df['First Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test ', flags=re.I, regex=True) | 
        zoho_df['Last Name'].str.contains(' test$', flags=re.I, regex=True)
    ]

count_leads_test = test_leads.shape[0]

# test_leads.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\test_leads_2.xlsx', index=False)
test_leads

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Mobile


In [81]:
zoho_df.drop(test_leads.index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_dedup_test = zoho_df.shape[0]

In [82]:
# dnm_leads = zoho_df.loc[(zoho_df['Do Not Mail'] == 'true') | (zoho_df['Do Not Contact'] == 'true')]
# count_dnm = dnm_leads.shape[0]

In [83]:
# zoho_df.drop(dnm_leads.index, inplace=True)
# zoho_df.reset_index(drop=True, inplace=True)

# count_after_dedup_dnm = zoho_df.shape[0]

In [84]:
# Cek Email OTA
email_OTA = m.find_OTA(zoho_df, 'Email')
count_email_OTA = email_OTA.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_OTA

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Mobile


In [85]:
# delete email OTA
count_after_clean_OTA = m.delete_OTA(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Mobile
0,zcrm_3584690000475129897,2024-01-02 03:06:22,2024-01-01 00:00:00,Bethany Roberts,Bethany Roberts,broberts2611@gmail.com,447802613306,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
1,zcrm_3584690000475363902,2024-01-03 23:45:38,2024-01-03 00:00:00,Emmy,Emmy,emmybadham@btinternet.com,447817178812,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
2,zcrm_3584690000475525104,2024-01-05 06:03:36,2024-01-04 00:00:00,Tikeya Holloway,Tikeya Holloway,tikeyaholloway@gmail.com,447582540744,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
3,zcrm_3584690000475544144,2024-01-05 01:52:12,2024-01-04 00:00:00,Jason And Emma,Jason And Emma,jasesanders1993@yahoo.com,447940033370,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
4,zcrm_3584690000475544281,2024-01-05 06:32:32,2024-01-04 00:00:00,Ben Caldwell,Ben Caldwell,caldwellb395@gmail.com,447934857239,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,empty,empty,empty,M,-,marina.atkins@hotmail.co.uk,447961502336,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1478,empty,empty,empty,Samantha,-,sam.laister5@gmail.com,447912877649,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1479,empty,empty,empty,Spencer,Hughes-Morris,spencer.d.hughes.morris@gmail.com,447826640590,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1480,empty,empty,empty,David,Ortega,me@davidortega.co.uk,447776552414,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty


In [86]:
# Cek Email Karma
email_karma = m.find_karma(zoho_df, 'Email')
count_email_karma = email_karma.shape[0]
# email_OTA.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\email_OTA.xlsx', index=False)

email_karma

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Mobile


In [87]:
# delete email Karma
count_after_clean_karma = m.delete_karma(zoho_df, 'Email')
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\after_clean_OTA.xlsx', index=False)

zoho_df

,Record Id,Created Time,Created Date,First Name,Last Name,Email,Phone,Country,Lead Brand,Lead Sub-Brand,Lead Source Description,Lead Source,Lead Locations,Website,Opt In,Agree with Terms and Condition,GDPR Compliant,Contact type,Mobile
0,zcrm_3584690000475129897,2024-01-02 03:06:22,2024-01-01 00:00:00,Bethany Roberts,Bethany Roberts,broberts2611@gmail.com,447802613306,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
1,zcrm_3584690000475363902,2024-01-03 23:45:38,2024-01-03 00:00:00,Emmy,Emmy,emmybadham@btinternet.com,447817178812,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
2,zcrm_3584690000475525104,2024-01-05 06:03:36,2024-01-04 00:00:00,Tikeya Holloway,Tikeya Holloway,tikeyaholloway@gmail.com,447582540744,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
3,zcrm_3584690000475544144,2024-01-05 01:52:12,2024-01-04 00:00:00,Jason And Emma,Jason And Emma,jasesanders1993@yahoo.com,447940033370,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
4,zcrm_3584690000475544281,2024-01-05 06:32:32,2024-01-04 00:00:00,Ben Caldwell,Ben Caldwell,caldwellb395@gmail.com,447934857239,United Kingdom,Karma Resorts,Other,Karma Salford Hall -Download Wedding Brochure ...,Web Enquiry,Karma Salford Hall,https://karmagroup.com/weddings-at-karma-salfo...,True,True,True,Email and Phone,empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1477,empty,empty,empty,M,-,marina.atkins@hotmail.co.uk,447961502336,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1478,empty,empty,empty,Samantha,-,sam.laister5@gmail.com,447912877649,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1479,empty,empty,empty,Spencer,Hughes-Morris,spencer.d.hughes.morris@gmail.com,447826640590,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty
1480,empty,empty,empty,David,Ortega,me@davidortega.co.uk,447776552414,empty,empty,empty,Karma Salford Hall Database (JUNE 2025) - Wedd...,Past Guests,Karma Salford Hall,empty,True,True,True,empty,empty


In [88]:
find_member_booking = zoho_df.copy()

find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1'
find_member_booking.loc[find_member_booking['Phone'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Phone1']), 'Same Value Find'] = 'Booking-Phone1' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_booking['Mobile']), 'Same Value Find'] = 'Booking-Mobile'
find_member_booking.loc[find_member_booking['Email'].isin(df_booking['Email']), 'Same Value Find'] = 'Booking-Email'

find_member_booking.loc[find_member_booking['Phone'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['Mobile'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'

find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

count_booking_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Booking', flags=re.I, regex=True)].shape[0]
count_member_dedup = find_member_booking.loc[find_member_booking['Same Value Find'].str.contains('Member', flags=re.I, regex=True)].shape[0]

find_member_booking.replace('empty', '', inplace=True)
# find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.csv', index=False)
# find_member_booking.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\booking-member-dedup.xlsx', index=False)

#find_member_booking.loc[find_member_booking['Same Value Find'] == 'Booking-Phone1'] 

In [89]:
# data phone, mobile dam email member yang ada di master data 
count_wash_member = zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email'])| 

    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

].shape[0]

zoho_df.drop(zoho_df.loc[
    zoho_df['Phone'].isin(df_booking['Phone1']) | 
    zoho_df['Phone'].isin(df_booking['Mobile']) |
    zoho_df['Mobile'].isin(df_booking['Phone1']) | 
    zoho_df['Mobile'].isin(df_booking['Mobile']) |
    zoho_df['Email'].isin(df_booking['Email']) |
    
    zoho_df['Phone'].isin(df_phone['Mobile']) | 
    zoho_df['Mobile'].isin(df_phone['Mobile']) |

    zoho_df['Email'].isin(df_email['Email'])

    ].index, inplace=True)
    
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_member = zoho_df.shape[0]


In [90]:
count_email_only = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty'))].shape[0]
count_phone_only = zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]
count_email_and_phone = zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty'))].shape[0]

count_blank = zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].shape[0]

# zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\blank_data.xlsx', index=False)

zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] == 'empty') & (zoho_df['Mobile'] == 'empty')), 'Contact Type'] = 'Email Only'
zoho_df.loc[(zoho_df['Email'] == 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Phone Only'
zoho_df.loc[(zoho_df['Email'] != 'empty') & ((zoho_df['Phone'] != 'empty') | (zoho_df['Mobile'] != 'empty')), 'Contact Type'] = 'Email and Phone'

zoho_df.drop(zoho_df.loc[(zoho_df['Email'] =='empty') & (zoho_df['Phone']=='empty') & (zoho_df['Mobile']=='empty')].index, inplace=True)
zoho_df.reset_index(drop=True, inplace=True)

count_after_wash_blank = zoho_df.shape[0]

In [91]:
arr_dedup = [count_deleted_wrong_format, count_duplicate_general, count_leads_test, count_email_OTA, count_email_karma, count_blank]
count_sum_dedup = sum(arr_dedup)
count_sum_dedup

0

In [92]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Filter Leads Awal : "+str(count_filter_awal))
print("Jumlah Format Email Salah : "+str(count_deleted_wrong_format))
print("Jumlah Setelah Clean Format Email Salah : "+str(count_after_deleted_wrong_format))
print("Jumlah Duplicate (Phone, Mobile dan Email) : "+str(count_duplicate_general))
print("Jumlah Setelah Clean Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Lead Test : "+str(count_leads_test))
print("Jumlah Setelah Clean Lead Test : "+str(count_after_dedup_test))
# print("Jumlah Lead Do Not Mail : "+str(count_dnm))
# print("Jumlah Setelah Clean Lead Do Not Mail : "+str(count_after_dedup_dnm))
print("Jumlah Email OTA : "+str(count_email_OTA))
print("Jumlah Setelah Clean Email OTA : "+str(count_after_clean_OTA))
print("Jumlah Email Karma : "+str(count_email_karma))
print("Jumlah Setelah Clean Email Karma : "+str(count_after_clean_karma))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Setelah Wash Member : "+str(count_after_wash_member))
print("Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : "+str(count_blank))
print("Jumlah Setelah Wash Phone Email Blank : "+str(count_after_wash_blank))
print("==========================================================")
print("Contact Type")
print("==========================================================")
print("Jumlah Email Only : "+str(count_email_only))
print("Jumlah Phone Only : "+str(count_phone_only))
print("Jumlah Email dan Phone : "+str(count_email_and_phone))

Summary
Jumlah Filter Leads Awal : 1482
Jumlah Format Email Salah : 0
Jumlah Setelah Clean Format Email Salah : 1482
Jumlah Duplicate (Phone, Mobile dan Email) : 0
Jumlah Setelah Clean Duplicate : 1482
Jumlah Lead Test : 0
Jumlah Setelah Clean Lead Test : 1482
Jumlah Email OTA : 0
Jumlah Setelah Clean Email OTA : 1482
Jumlah Email Karma : 0
Jumlah Setelah Clean Email Karma : 1482
Jumlah Wash Member : 5
Jumlah Wash Booking : 0
Jumlah Setelah Wash Member : 1477
Jumlah Phone Email Blank (Setelah phone yang tidak valid dihapus) : 0
Jumlah Setelah Wash Phone Email Blank : 1477
Contact Type
Jumlah Email Only : 6
Jumlah Phone Only : 0
Jumlah Email dan Phone : 1471


In [93]:
print("==========================================================")
print("Summary Report")
print("==========================================================")
print("Jumlah Data Awal : "+str(count_filter_awal))
print("Jumlah Wash Dedup : "+str(count_sum_dedup))
print("Jumlah Wash Member : "+str(count_member_dedup))
print("Jumlah Wash Booking : "+str(count_booking_dedup))
print("Jumlah Data Akhir : "+str(count_after_wash_blank))

Summary Report
Jumlah Data Awal : 1482
Jumlah Wash Dedup : 0
Jumlah Wash Member : 5
Jumlah Wash Booking : 0
Jumlah Data Akhir : 1477


In [94]:
zoho_df.replace('empty', '', inplace=True, regex=True)

zoho_df['First Name'] = zoho_df['First Name'].str.title()
zoho_df['Last Name'] = zoho_df['Last Name'].str.title()

zoho_df.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\KSH.csv', index=False)
# zoho_df.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\newsletter-clean.xlsx', index=False)